# Herschel Stripe 82 master catalogue
## Preparation of SDSS Stripe 82 data

This catalogue comes from `dmu0_SDSS-S82`.

In the catalogue, we keep:

- The `object_id` as unique object identifier;
- The position;
- The u, g, r, i, z, aperture magnitude (for now in 3”);
- The u, g, r, i, z, auto fluxes and magnitudes.


We don't know when the maps have been observed. We will use the year of the reference paper.

In [ ]:
from herschelhelp_internal import git_version
print("This notebook was run with herschelhelp_internal version: \n{}".format(git_version()))

In [ ]:
%matplotlib inline
#%config InlineBackend.figure_format = 'svg'

import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))

from collections import OrderedDict
import os

from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.table import Column, Table
import numpy as np

from herschelhelp_internal.flagging import  gaia_flag_column
from herschelhelp_internal.masterlist import nb_astcor_diag_plot, remove_duplicates
from herschelhelp_internal.utils import astrometric_correction, mag_to_flux

In [ ]:
OUT_DIR =  os.environ.get('TMP_DIR', "./data_tmp")
try:
    os.makedirs(OUT_DIR)
except FileExistsError:
    pass

RA_COL = "sdss_ra"
DEC_COL = "sdss_dec"

## I - Column selection

In [ ]:
imported_columns = OrderedDict({
        "objID": "sdss_id",
        "ra": "sdss_ra",
        "dec": "sdss_dec",
        "probPSF": "sdss_stellarity",
        "petroMag_u": "m_sdss_u",
        "petroMag_g": "m_sdss_g",
        "petroMag_r": "m_sdss_r",
        "petroMag_i": "m_sdss_i",
        "petroMag_z": "m_sdss_z",
        "petroMagErr_u": "merr_sdss_u",
        "petroMagErr_g": "merr_sdss_g",
        "petroMagErr_r": "merr_sdss_r",
        "petroMagErr_i": "merr_sdss_i",
        "petroMagErr_z": "merr_sdss_z",
        "fiberMag_u": "m_ap_sdss_u",
        "fiberMag_g": "m_ap_sdss_g",
        "fiberMag_r": "m_ap_sdss_r",
        "fiberMag_i": "m_ap_sdss_i",
        "fiberMag_z": "m_ap_sdss_z",
        "fiberMagErr_u": "merr_ap_sdss_u",
        "fiberMagErr_g": "merr_ap_sdss_g",
        "fiberMagErr_r": "merr_ap_sdss_r",
        "fiberMagErr_i": "merr_ap_sdss_i",
        "fiberMagErr_z": "merr_ap_sdss_z"        
    })


catalogue = Table.read("../../dmu0/dmu0_SDSS-S82/data/SDSS-S82_HELP-coverage.fits")[list(imported_columns)]
for column in imported_columns:
    catalogue[column].name = imported_columns[column]

epoch = 2013

# Clean table metadata
catalogue.meta = None

In [ ]:
# Adding flux and band-flag columns
for col in catalogue.colnames:
    if col.startswith('m_'):
        
        errcol = "merr{}".format(col[1:])
        flux, error = mag_to_flux(np.array(catalogue[col]), np.array(catalogue[errcol]))
        
        # Fluxes are added in µJy
        catalogue.add_column(Column(flux * 1.e6, name="f{}".format(col[1:])))
        catalogue.add_column(Column(error * 1.e6, name="f{}".format(errcol[1:])))
        
        # Band-flag column
        if 'ap' not in col:
            catalogue.add_column(Column(np.zeros(len(catalogue), dtype=bool), name="flag{}".format(col[1:])))
        


In [ ]:
catalogue[:10].show_in_notebook()

## II - Removal of duplicated sources

We remove duplicated objects from the input catalogues.

In [ ]:
SORT_COLS = []
FLAG_NAME = 'sdss_flag_cleaned'

nb_orig_sources = len(catalogue)

catalogue = remove_duplicates(
    catalogue, RA_COL, DEC_COL, 
    sort_col= SORT_COLS,
    flag_name=FLAG_NAME)

nb_sources = len(catalogue)

print("The initial catalogue had {} sources.".format(nb_orig_sources))
print("The cleaned catalogue has {} sources ({} removed).".format(nb_sources, nb_orig_sources - nb_sources))
print("The cleaned catalogue has {} sources flagged as having been cleaned".format(np.sum(catalogue[FLAG_NAME])))

## III - Astrometry correction

We match the astrometry to the Gaia one. We limit the Gaia catalogue to sources with a g band flux between the 30th and the 70th percentile. Some quick tests show that this give the lower dispersion in the results.

In [ ]:
gaia = Table.read("../../dmu0/dmu0_GAIA/data/GAIA_Herschel-Stripe-82.fits")
gaia_coords = SkyCoord(gaia['ra'], gaia['dec'])

In [ ]:
nb_astcor_diag_plot(catalogue[RA_COL], catalogue[DEC_COL], 
                    gaia_coords.ra, gaia_coords.dec, near_ra0=True)

In [ ]:
delta_ra, delta_dec =  astrometric_correction(
    SkyCoord(catalogue[RA_COL], catalogue[DEC_COL]),
    gaia_coords, near_ra0=True
)

print("RA correction: {}".format(delta_ra))
print("Dec correction: {}".format(delta_dec))

In [ ]:
catalogue[RA_COL] +=  delta_ra.to(u.deg)
catalogue[DEC_COL] += delta_dec.to(u.deg)

In [ ]:
nb_astcor_diag_plot(catalogue[RA_COL], catalogue[DEC_COL], 
                    gaia_coords.ra, gaia_coords.dec, near_ra0=True)

## IV - Flagging Gaia objects

In [ ]:
catalogue.add_column(
    gaia_flag_column(SkyCoord(catalogue[RA_COL], catalogue[DEC_COL]), epoch, gaia)
)

In [ ]:
GAIA_FLAG_NAME = "sdss_flag_gaia"

catalogue['flag_gaia'].name = GAIA_FLAG_NAME
print("{} sources flagged.".format(np.sum(catalogue[GAIA_FLAG_NAME] > 0)))

## V - Flagging objects near bright stars

# VI - Saving to disk

In [ ]:
catalogue.write("{}/SDSS-S82.fits".format(OUT_DIR), overwrite=True)